In [1]:
import pandas as pd
import plotly.express as px
import geopandas as gpd

## Pandas and GeoJson Prep

In [2]:
def GetDataframe():
    df_Raw = pd.read_csv('../Datasets/Family Income and Expenditure.csv')

    #Remove whitespaces like in ' ARMM'
    df_Raw['Region'] = df_Raw['Region'].apply(lambda x: x.strip())

    #Change the food column name
    df_Raw.rename(columns={
                    'Bread and Cereals Expenditure':'Bread and Cereals',
                    'Meat Expenditure':'Meat',
                    'Total Rice Expenditure':'Rice', 
                    'Total Fish and  marine products Expenditure':'Seafood', 
                    'Fruit Expenditure':'Fruits', 
                    'Vegetables Expenditure':'Vegetables'
                  },
                  inplace=True
                 )

    df_Raw['Household Head Education Level'] = "None"
    def SetEduLevel(ref_Dataframe, str_ScanFor, str_Replace):
        for currStr in str_ScanFor:
            ref_Dataframe.loc[ref_Dataframe['Household Head Highest Grade Completed'].str.contains(currStr), 'Household Head Education Level'] = str_Replace


    SetEduLevel(df_Raw, ['No Grade Completed'], 'None')
    SetEduLevel(df_Raw, ['Preschool'], 'Preschool')
    SetEduLevel(df_Raw, ['Grade 1', 'Grade 2','Grade 3','Grade 4','Grade 5','Grade 6','Elementary Graduate'], 'Primary')
    SetEduLevel(df_Raw, ['High School'], 'Secondary')
    SetEduLevel(df_Raw, ['Program'], 'TESDA')
    SetEduLevel(df_Raw, ['College', 'Post Secondary', 'Baccalaureate', 'Business and Administration Programs', 'Humanities Programs', 'Engineering and Engineering Trades Programs','Social and Behavioral Science Programs','Health Programs','Engineering and Engineering trades Programs','Computing/Information Technology Programs','Mathematics and Statistics Programs','Law Programs','Journalism and Information Programs','Architecture and Building Programs','Manufacturing and Processing Programs', 'Life Sciences Programs','Physical Sciences Programs', 'Arts Programs','Veterinary Programs'], 
                'Tertiary')


    def SetRegionID(ref_Dataframe):
        RegionID = GetRegionIDs()
        for x in RegionID.keys():
            ref_Dataframe.loc[ref_Dataframe['Region'] == x, 'RegionID'] = RegionID[x]

    SetRegionID(df_Raw)
    
    return df_Raw

In [3]:
def GetColumnNames_ForFood():
    return ['Bread and Cereals','Meat', 'Rice', 'Seafood', 'Fruits', 'Vegetables']

In [4]:
def GetGeoJson():
    gj_PhMap = gpd.read_file('../Datasets/country.0.001.json')

    # MATCH REGION NAME TO OUR CSV 
    gj_PhMap['adm1_en'] = gj_PhMap['adm1_en'].replace(
        ['Region I (Ilocos Region)', 'Region II (Cagayan Valley)',
           'Region III (Central Luzon)', 'Region IV-A (CALABARZON)',
           'Region V (Bicol Region)', 'Region VI (Western Visayas)',
           'Region VII (Central Visayas)', 'Region VIII (Eastern Visayas)',
           'Region IX (Zamboanga Peninsula)', 'Region X (Northern Mindanao)',
           'Region XI (Davao Region)', 'Region XII (SOCCSKSARGEN)',
           'National Capital Region (NCR)',
           'Cordillera Administrative Region (CAR)', 'Region XIII (Caraga)',
           'MIMAROPA Region',
           'Bangsamoro Autonomous Region In Muslim Mindanao (BARMM)']
        ,
    
        ['I - Ilocos Region', 'II - Cagayan Valley',
         'III - Central Luzon', 'IVA - CALABARZON',
         'V - Bicol Region', 'VI - Western Visayas', 
         'VII - Central Visayas', 'VIII - Eastern Visayas',
         'IX - Zasmboanga Peninsula', 'X - Northern Mindanao',
         'XI - Davao Region', 'XII - SOCCSKSARGEN',
         'NCR',
         'CAR', 'Caraga',
         'IVB - MIMAROPA', 
         'ARMM'])
    return gj_PhMap

# All Graphs

## Choropleth of Income

In [5]:
def GetFig_Choropleth(ref_Dataframe, ref_GeoJson, agg_Mode):
    match agg_Mode:
        case 'Max':
           agg_Income = ref_Dataframe.groupby('Region')['Total Household Income'].max()
        case 'Min':
           agg_Income = ref_Dataframe.groupby('Region')['Total Household Income'].min()
        case 'Median':
           agg_Income = ref_Dataframe.groupby('Region')['Total Household Income'].median()
        case default:
           agg_Income = ref_Dataframe.groupby('Region')['Total Household Income'].mean()
    
    
    fig = px.choropleth_mapbox(
        data_frame = agg_Income,
        geojson = ref_GeoJson,
        locations = agg_Income.index, #'Region',
        featureidkey = 'properties.adm1_en',
        color = agg_Income.values, #'Total Household Income',
        center = {'lat': 12.738500, 'lon': 121.766632},
        mapbox_style= 'carto-positron',
        zoom = 4,
        opacity = 0.3,
        height=600,
        labels={'color':'Php'},
        title='Income per Region'
    )
    return fig

## Pie Chart of Top 10 Jobs

In [6]:
def GetFig_MostEmployedJobs(ref_Dataframe, region_Name):
    df_AllJobs = ref_Dataframe[ref_Dataframe['Region'] == region_Name]['Household Head Occupation'].value_counts()
    df_Top10Jobs = df_AllJobs[:10]
    #df_Top10Jobs['Others'] = df_AllJobs[10:].sum()
    #print(df_Top10Jobs)
    #fig = px.histogram(data_frame=df_Top10Jobs, x=df_Top10Jobs.index, y= df_Top10Jobs.values)
    
    fig = px.pie(df_Top10Jobs, values = df_Top10Jobs.values, names = df_Top10Jobs.index, title='Top 10 Most Employed Jobs in ' + region_Name, hover_name=df_Top10Jobs.index)
    return fig

## Stacked Barchart of Food Breakdown per Region

In [7]:
def GetFig_FoodBreakdown(ref_Dataframe, selected_Foods, grouping_By):
    avg_FoodExpenses = ref_Dataframe.groupby(grouping_By)[selected_Foods].mean()
    
    fig = px.bar(data_frame = avg_FoodExpenses, x=avg_FoodExpenses.index, y=selected_Foods, 
                 title='Average Food Expenses' , labels={'value':'Php', 'variable':'Food Expenses'}) 
    fig.update_layout(barmode='stack')
    return fig

## Stacked Barchart of Income vs Expenses

In [8]:
def GetFig_IncomeVsExpenses(ref_Dataframe, selected_Regions, selected_Category, is_Collapse):
    #ref_Dataframe = GetDataframe() 
    #selected_Regions = ['NCR', 'CAR']
    #selected_Category = ['Income', 'Utilities']
    #is_Collapse = False
    
    queryCols =['Region']
    for i in selected_Category:
        for j in GetExpenseTypes()[i]:
            queryCols.append(j)
    
    df_IncomeAndExpenses = ref_Dataframe[queryCols].copy()
    df_IncomeAndExpenses.set_index('Region', inplace=True)
    
    if (is_Collapse):
        for i in selected_Category:
            df_IncomeAndExpenses[i] = df_IncomeAndExpenses[GetExpenseTypes()[i]].sum(axis=1)
            df_IncomeAndExpenses.drop(columns=GetExpenseTypes()[i], inplace=True)
    
    aggByRegion = df_IncomeAndExpenses.groupby('Region')[df_IncomeAndExpenses.columns].median()
    
    # FILTER TO ONLY OUR SELECTED REGIONS
    aggByRegion = aggByRegion.loc[selected_Regions]
    
    # MAKE EXPENSES NEGATIVE
    aggByRegion.loc[:, ~aggByRegion.columns.isin(['Income', 'Total Household Income'])] *= -1
    
    # PLOT THE GRAPH
    fig = px.bar(data_frame = aggByRegion, x=aggByRegion.index, y=aggByRegion.columns, barmode='relative',
                 title = 'Median Income and Expenses per Region',
                 height = 600,
                 labels={
                    'value' : 'Php',
                    'variable' : 'Income and Expenses'
                })
    return fig
    
    
    
    # selected_Category = ['Income', 'Utilities']
    
    # queryCols =['Region']
    # for i in selected_Category:
    #     for j in GetExpenseTypes()[i]:
    #         queryCols.append(j)
    
    # df_IncomeAndExpenses = ref_Dataframe[queryCols].copy()
    
    # # GET THE MEDIAN OF THE REGIONS
    # queryCols.remove('Region')
    # aggByRegion = df_IncomeAndExpenses.groupby('Region')[queryCols].median().reset_index()
    
    # # FILTER TO ONLY OUR SELECTED REGIONS
    # aggByRegion = aggByRegion[aggByRegion['Region'].apply(lambda x: x in selected_Regions)]
    
    # # MAKE EXPENSES NEGATIVE
    # aggByRegion[queryCols] *= -1
    # aggByRegion['Total Household Income'] *= -1

    # # PLOT THE GRAPH
    # fig = px.bar(data_frame = aggByRegion, x='Region', y=queryCols, barmode='relative',
    #              title = 'Median Income and Expenses per Region',
    #              height = 600,
    #              labels={
    #                 'value' : 'Php',
    #                 'variable' : 'Income and Expenses'
    #             })
    # #fig.update_layout(xaxis={'categoryorder':'total descending'})
    # return fig

In [9]:
def GetGroupings_ForFoodBreakdown():
    return [ 'Household Head Sex',
       'Household Head Age', 'Household Head Marital Status',
       'Household Head Education Level',
       'Household Head Job or Business Indicator', 
       'Household Head Class of Worker', 'Type of Household',
       'Total Number of Family members',
       'Members with age less than 5 year old',
       'Members with age 5 - 17 years old',
       'Total number of family members employed', 'Type of Building/House',
       'Type of Roof', 'Type of Walls', 'House Floor Area', 'House Age',
       'Number of bedrooms', 'Tenure Status', 'Toilet Facilities',
       'Electricity', 'Main Source of Water Supply', 'Number of Television',
       'Number of CD/VCD/DVD', 'Number of Component/Stereo set',
       'Number of Refrigerator/Freezer', 'Number of Washing Machine',
       'Number of Airconditioner', 'Number of Car, Jeep, Van',
       'Number of Landline/wireless telephones', 'Number of Cellular phone',
       'Number of Personal Computer', 'Number of Stove with Oven/Gas Range',
       'Number of Motorized Banca', 'Number of Motorcycle/Tricycle']

In [10]:
def GetExpenseTypes(): 
    return {
            "Income": ['Total Household Income'],
            "Food": ['Total Food Expenditure'],
            "Education": ['Education Expenditure'],
            "Healthcare": ['Medical Care Expenditure'],
            "Clothing": ['Clothing, Footwear and Other Wear Expenditure'],
            "Utilities":['Housing and water Expenditure', 'Imputed House Rental Value', 'Communication Expenditure','Miscellaneous Goods and Services Expenditure','Transportation Expenditure','Crop Farming and Gardening expenses'],
            "Social" : ['Special Occasions Expenditure', 'Restaurant and hotels Expenditure'],
            "Vices": ['Tobacco Expenditure', 'Alcoholic Beverages Expenditure']
            }

In [11]:
def GetRegionIDs():
    return {
            'I - Ilocos Region' : 1, 
            'CAR' : 1.5,
            'II - Cagayan Valley' : 2,
            'III - Central Luzon' : 3,
            'NCR': 3.5,
            'IVA - CALABARZON' : 4,
            'IVB - MIMAROPA' : 4.5, 
            'V - Bicol Region' : 5,
            'VI - Western Visayas' : 6,
            'VII - Central Visayas' : 7,
            'VIII - Eastern Visayas' : 8, 
            'IX - Zasmboanga Peninsula' : 9,
            'X - Northern Mindanao' : 10, 
            'XI - Davao Region' : 11,
            'XII - SOCCSKSARGEN' : 12,
            'Caraga' : 13,  
            'ARMM' : 14
        }

In [12]:
# ref_Dataframe = GetDataframe() 
# selected_Regions = ['NCR', 'CAR']
# selected_Category = ['Income', 'Utilities']
# is_Collapse = False

# queryCols =['Region']
# for i in selected_Category:
#     for j in GetExpenseTypes()[i]:
#         queryCols.append(j)

# df_IncomeAndExpenses = ref_Dataframe[queryCols].copy()
# df_IncomeAndExpenses.set_index('Region', inplace=True)

# # GET THE MEDIAN OF THE REGIONS
# queryCols.remove('Region')

# if (is_Collapse):
#     for i in selected_Category:
#         df_IncomeAndExpenses[i] = df_IncomeAndExpenses[GetExpenseTypes()[i]].sum(axis=1)
#         df_IncomeAndExpenses.drop(columns=GetExpenseTypes()[i], inplace=True)

# aggByRegion = df_IncomeAndExpenses.groupby('Region')[df_IncomeAndExpenses.columns].median()

# # FILTER TO ONLY OUR SELECTED REGIONS
# aggByRegion = aggByRegion.loc[selected_Regions]

# # MAKE EXPENSES NEGATIVE
# aggByRegion.loc[:, ~aggByRegion.columns.isin(['Income', 'Total Household Income'])] *= -1

# # PLOT THE GRAPH
# fig = px.bar(data_frame = aggByRegion, x=aggByRegion.index, y=aggByRegion.columns, barmode='relative',
#              title = 'Median Income and Expenses per Region',
#              height = 600,
#              labels={
#                 'value' : 'Php',
#                 'variable' : 'Income and Expenses'
#             })
# return fig